In [1]:
import imageio
import time
import math
import cv2
import numpy as np
from numpy import uint8

In [2]:
from tkinter import filedialog
from tkinter.filedialog import asksaveasfile
from tkinter import *
from PIL import *
import os
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk,Image

In [3]:
#ToDo...This checks if the file selected is an image or not
#def checkIfImg(imgLoc):

In [4]:
def extractImgData(imgLoc):
    pixelMatrix = imageio.imread(imgLoc) #use imsave(filename, pixelMatrix) to save the image
    return pixelMatrix

In [5]:
def grayscale(height, width, pixelMatrix):
    for i in range(height):
        for j in range(width):
            avgCol = ((int)(pixelMatrix[i,j,0]) + (int)(pixelMatrix[i,j,1]) + (int)(pixelMatrix[i,j,2]))/3
            pixelMatrix[i,j,0] = pixelMatrix[i,j,1] = pixelMatrix[i,j,2] = uint8(avgCol)

In [6]:
def vertical_flip(height, width, pixelMatrix):
    i = 0
    while (i < height):
        j = 0
        while (j < width/2):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[i,width-1-j,0]
            pixelMatrix[i,width-1-j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[i,width-1-j,1]
            pixelMatrix[i,width-1-j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[i,width-1-j,2]
            pixelMatrix[i,width-1-j,2] = blueTemp
            
            j += 1
        i += 1

In [7]:
def horizontal_flip(height,width,pixelMatrix):
    i = 0
    while (i < height/2):
        j = 0
        while (j < width):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[height-1-i,j,0]
            pixelMatrix[height-1-i,j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[height-1-i,j,1]
            pixelMatrix[height-1-i,j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[height-1-i,j,2]
            pixelMatrix[height-1-i,j,2] = blueTemp
            
            j += 1
        i += 1

In [8]:
def init_surr_cells(pixel):
    surr_cells = [[[0,0,0],[0,0,0],[0,0,0]],[[0,0,0],[pixel[0], pixel[1], pixel[2]],[0,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]
    return surr_cells

In [9]:
def Sobel_Operator(surr_cells, color):
    GX = [[-1,0,1],[-2,0,2],[-1,0,1]]
    GY = [[-1,-2,-1],[0,0,0],[1,2,1]]
    val_gx = val_gy = 0
    if color == 'r':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][0]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][0]*GY[i][j]
    elif color == 'g':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][1]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][1]*GY[i][j]
    else:
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][2]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][2]*GY[i][j]
    net_val = round(math.sqrt(val_gx**2 + val_gy**2))
    if net_val > 255:
        net_val = 255
    return net_val

In [10]:
def edgify(height,width,pixelMatrix):
    temp = list()
    i = 0
    while i < height:
        j = 0
        while j < width:
            surr_cells = init_surr_cells(pixelMatrix[i,j])
            
            if i != height - 1:
                surr_cells[2][1][0] = pixelMatrix[i+1,j,0]
                surr_cells[2][1][1] = pixelMatrix[i+1,j,1]
                surr_cells[2][1][2] = pixelMatrix[i+1,j,2]
                if j != width - 1:
                    surr_cells[2][2][0] = pixelMatrix[i+1,j+1,0]
                    surr_cells[2][2][1] = pixelMatrix[i+1,j+1,1]
                    surr_cells[2][2][2] = pixelMatrix[i+1,j+1,2]
                if j != 0:
                    surr_cells[2][0][0] = pixelMatrix[i+1,j-1,0]
                    surr_cells[2][0][1] = pixelMatrix[i+1,j-1,1]
                    surr_cells[2][0][2] = pixelMatrix[i+1,j-1,2]
                 
            if i != 0:
                surr_cells[0][1][0] = pixelMatrix[i-1,j,0]
                surr_cells[0][1][1] = pixelMatrix[i-1,j,1]
                surr_cells[0][1][2] = pixelMatrix[i-1,j,2]
                if j != width - 1:
                    surr_cells[0][2][0] = pixelMatrix[i-1,j+1,0]
                    surr_cells[0][2][1] = pixelMatrix[i-1,j+1,1]
                    surr_cells[0][2][2] = pixelMatrix[i-1,j+1,2]
                if j != 0:
                    surr_cells[0][0][0] = pixelMatrix[i-1,j-1,0]
                    surr_cells[0][0][1] = pixelMatrix[i-1,j-1,1]
                    surr_cells[0][0][2] = pixelMatrix[i-1,j-1,2]
                 
            if j != width - 1:
                surr_cells[1][2][0] = pixelMatrix[i,j+1,0]
                surr_cells[1][2][1] = pixelMatrix[i,j+1,1]
                surr_cells[1][2][2] = pixelMatrix[i,j+1,2]
             
            if j != 0:
                surr_cells[1][0][0] = pixelMatrix[i,j-1,0]
                surr_cells[1][0][1] = pixelMatrix[i,j-1,1]
                surr_cells[1][0][2] = pixelMatrix[i,j-1,2]
            x = Sobel_Operator(surr_cells, 'r') 
            y = Sobel_Operator(surr_cells, 'g') 
            z = Sobel_Operator(surr_cells, 'b') 
            temp.append([x,y,z])
            j += 1
        count = (int)(i*100/height)
        i += 1
    i = 0
    k = 0
    while i < height:
        j = 0
        while j < width:
            pixelMatrix[i,j,0] = (uint8)(temp[k][0])
            pixelMatrix[i,j,1] = (uint8)(temp[k][1])
            pixelMatrix[i,j,2] = (uint8)(temp[k][2])
            k += 1
            j += 1
        i += 1

In [11]:
def solarization(height, width, pixelMatrix, threshold):
    i = 0
    while i < height:
        j = 0
        while j < width:
            if pixelMatrix[i,j,0] >= threshold:
                pixelMatrix[i,j,0] = 255 - pixelMatrix[i,j,0]
                
            if pixelMatrix[i,j,1] >= threshold:
                pixelMatrix[i,j,1] = 255 - pixelMatrix[i,j,1]
                
            if pixelMatrix[i,j,2] >= threshold:
                pixelMatrix[i,j,2] = 255 - pixelMatrix[i,j,2]
            j += 1
        i += 1

In [12]:
def edgify1(height,width,pixelMatrix):
    i = j = 0
    surr_cells = init_surr_cells(pixelMatrix[i,j])
            
    if i != height - 1:
        surr_cells[2][1] = pixelMatrix[i+1,j]
        if j != width - 1:
            surr_cells[2][2] = pixelMatrix[i+1,j+1]
        if j != 0:
            surr_cells[2][0] = pixelMatrix[i+1,j-1] 

    if i != 0:
        surr_cells[0][1] = pixelMatrix[i-1,j]
        if j != width - 1:
            surr_cells[0][2] = pixelMatrix[i-1,j+1]
        if j != 0:
            surr_cells[0][0] = pixelMatrix[i-1,j-1]

    if j != width - 1:
        surr_cells[1][2] = pixelMatrix[i,j+1]

    if j != 0:
        surr_cells[1][0] = pixelMatrix[i,j-1]
    '''
    if i == 0 and j == 0:
        print(surr_cells)
        print(surr_cells[1][1][0] + surr_cells[1][0][0])
    '''
    pixelMatrix[i,j,0] = Sobel_Operator(surr_cells, 'r') 
    pixelMatrix[i,j,1] = Sobel_Operator(surr_cells, 'g') 
    pixelMatrix[i,j,2] = Sobel_Operator(surr_cells, 'b')
    print(pixelMatrix[i,j])

In [13]:
def brightness(height, width, pixelMatrix,scalingFactor):
        i=0;
        j=0;
        while(i<height):
            j=0
            while(j<width):
                
                
                if(pixelMatrix[i,j,0]+scalingFactor>255):
                    pixelMatrix[i,j,0]=255
                else:
                     pixelMatrix[i,j,0]=pixelMatrix[i,j,0]+scalingFactor
                
                if(pixelMatrix[i,j,1]+scalingFactor>255):
                    pixelMatrix[i,j,1]=255
                else:
                    pixelMatrix[i,j,1]=pixelMatrix[i,j,1]+scalingFactor
                
                
                    
                if(pixelMatrix[i,j,2]+scalingFactor>255):
                    pixelMatrix[i,j,2]=255
                else:
                    pixelMatrix[i,j,2]=pixelMatrix[i,j,2]+scalingFactor
                
                j+=1
            i+=1

In [14]:
def contrast(height,width, pixelMatrix, contrastFactor):
    i=0
    j=0
    F=(259*(255+contrastFactor))/(255*(259-contrastFactor))

    while(i<height):
        j=0
        while(j<width):
            if((F*(pixelMatrix[i,j,0]-128)) + 128 >255):
                pixelMatrix[i,j,0]=255
            elif((F*(pixelMatrix[i,j,0]-128)) + 128 <0):
                pixelMatrix[i,j,0]=0
            else:
                pixelMatrix[i,j,0]=(F*(pixelMatrix[i,j,0]-128)) + 128 
            if((F*(pixelMatrix[i,j,1]-128)) + 128 >255):
                pixelMatrix[i,j,1]=255
            elif((F*(pixelMatrix[i,j,1]-128)) + 128 <0):
                pixelMatrix[i,j,1]=0
            else:
                pixelMatrix[i,j,1]=(F*(pixelMatrix[i,j,1]-128)) + 128
            if((F*(pixelMatrix[i,j,2]-128)) + 128 > 255):
                pixelMatrix[i,j,2]=255
            elif((F*(pixelMatrix[i,j,2]-128)) + 128 <0):
                pixelMatrix[i,j,2]=0
            else:
                pixelMatrix[i,j,2]=(F*(pixelMatrix[i,j,2]-128)) + 128
            j+=1
        i+=1
            

In [15]:
def blur(height, width, pixelMatrix, counter):
    for x in range(counter):
        temp=list()
        i=0
        while(i<height):
            j=0
            while(j<width):
                avgR = (int)(pixelMatrix[i,j,0])
                avgG = (int)(pixelMatrix[i,j,1])
                avgB = (int)(pixelMatrix[i,j,2])
                totalNeighbors=1

                if(i!=height-1):
                    avgR += (int)(pixelMatrix[i+1,j,0])
                    avgG += (int)(pixelMatrix[i+1,j,1])
                    avgB += (int)(pixelMatrix[i+1,j,2])
                    totalNeighbors+=1

                    if(j!=width-1):
                        avgR += (int)(pixelMatrix[i+1,j+1,0])
                        avgG += (int)(pixelMatrix[i+1,j+1,1])
                        avgB += (int)(pixelMatrix[i+1,j+1,2])
                        totalNeighbors+=1

                    if(j!=0):
                        avgR += (int)(pixelMatrix[i+1,j-1,0])
                        avgG += (int)(pixelMatrix[i+1,j-1,1])
                        avgB += (int)(pixelMatrix[i+1,j-1,2])
                        totalNeighbors+=1

                if(i!=0):
                    avgR += (int)(pixelMatrix[i-1,j,0])
                    avgG += (int)(pixelMatrix[i-1,j,1])
                    avgB += (int)(pixelMatrix[i-1,j,2])
                    totalNeighbors+=1

                    if(j!=width-1):
                        avgR += (int)(pixelMatrix[i-1,j+1,0])
                        avgG += (int)(pixelMatrix[i-1,j+1,1])
                        avgB += (int)(pixelMatrix[i-1,j+1,2])
                        totalNeighbors+=1

                    if(j!=0):
                        avgR += (int)(pixelMatrix[i-1,j-1,0])
                        avgG += (int)(pixelMatrix[i-1,j-1,1])
                        avgB += (int)(pixelMatrix[i-1,j-1,2])
                        totalNeighbors+=1

                if(j!=width-1):
                    avgR += (int)(pixelMatrix[i,j+1,0])
                    avgG += (int)(pixelMatrix[i,j+1,1])
                    avgB += (int)(pixelMatrix[i,j+1,2])
                    totalNeighbors+=1

                if(j!=0):
                    avgR += (int)(pixelMatrix[i,j-1,0])
                    avgG += (int)(pixelMatrix[i,j-1,1])
                    avgB += (int)(pixelMatrix[i,j-1,2])
                    totalNeighbors+=1
                avgR = round(avgR/totalNeighbors)
                avgB = round(avgB/totalNeighbors)
                avgG = round(avgG/totalNeighbors)
                if (avgR > 255):
                    avgR = 255
                if (avgB > 255):
                    avgB = 255
                if (avgG > 255):
                    avgG = 255
                temp.append([avgR,avgG,avgB])
                j=j+1
            i=i+1

        f=0
        g=0
        l=0
        while(f<height):
            g=0
            while(g<width):
                pixelMatrix[f][g][0]=(uint8)(temp[l][0])
                pixelMatrix[f][g][1]=(uint8)(temp[l][1])
                pixelMatrix[f][g][2]=(uint8)(temp[l][2])
                l=l+1
                g=g+1
            f=f+1

In [16]:
def vignette(height,width,pixelMatrix):
    kernelMatrix = cv2.getGaussianKernel(height,300)*cv2.getGaussianKernel(width,300).T
    normalizedKernelMatrix = kernelMatrix/np.linalg.norm(kernelMatrix)
    scale = 255
    maskMatrix = scale*normalizedKernelMatrix
    for x in range(3):
        pixelMatrix[:,:,x] = pixelMatrix[:,:,x]*maskMatrix

In [17]:
def blurv2(height, width, pixelMatrix):
    temp = np.zeros((height,width,3))
    i = 0
    #mean = np.array([[1,1,1],[1,1,1],[1,1,1]])
    while i < height:
        j = 0
        while j < width:
            surr_cells = init_surr_cells(pixelMatrix[i,j])
            
            if i != height - 1:
                surr_cells[2][1][0] = pixelMatrix[i+1,j,0]
                surr_cells[2][1][1] = pixelMatrix[i+1,j,1]
                surr_cells[2][1][2] = pixelMatrix[i+1,j,2]
                if j != width - 1:
                    surr_cells[2][2][0] = pixelMatrix[i+1,j+1,0]
                    surr_cells[2][2][1] = pixelMatrix[i+1,j+1,1]
                    surr_cells[2][2][2] = pixelMatrix[i+1,j+1,2]
                if j != 0:
                    surr_cells[2][0][0] = pixelMatrix[i+1,j-1,0]
                    surr_cells[2][0][1] = pixelMatrix[i+1,j-1,1]
                    surr_cells[2][0][2] = pixelMatrix[i+1,j-1,2]
                 
            if i != 0:
                surr_cells[0][1][0] = pixelMatrix[i-1,j,0]
                surr_cells[0][1][1] = pixelMatrix[i-1,j,1]
                surr_cells[0][1][2] = pixelMatrix[i-1,j,2]
                if j != width - 1:
                    surr_cells[0][2][0] = pixelMatrix[i-1,j+1,0]
                    surr_cells[0][2][1] = pixelMatrix[i-1,j+1,1]
                    surr_cells[0][2][2] = pixelMatrix[i-1,j+1,2]
                if j != 0:
                    surr_cells[0][0][0] = pixelMatrix[i-1,j-1,0]
                    surr_cells[0][0][1] = pixelMatrix[i-1,j-1,1]
                    surr_cells[0][0][2] = pixelMatrix[i-1,j-1,2]
                 
            if j != width - 1:
                surr_cells[1][2][0] = pixelMatrix[i,j+1,0]
                surr_cells[1][2][1] = pixelMatrix[i,j+1,1]
                surr_cells[1][2][2] = pixelMatrix[i,j+1,2]
             
            if j != 0:
                surr_cells[1][0][0] = pixelMatrix[i,j-1,0]
                surr_cells[1][0][1] = pixelMatrix[i,j-1,1]
                surr_cells[1][0][2] = pixelMatrix[i,j-1,2]
            avgR = avgG = avgB = 0
            for k in range(3):
                for l in range(3):
                    avgR += surr_cells[k][l][0]
                    avgG += surr_cells[k][l][1]
                    avgB += surr_cells[k][l][2]
            avgR = round(avgR/9)
            avgB = round(avgB/9)
            avgG = round(avgG/9)
            if (avgR > 255):
                avgR = uint8(255)
            if (avgB > 255):
                avgB = uint8(255)
            if (avgG > 255):
                avgG = uint8(255)
            temp[i,j] = [avgR,avgG,avgB]
            j += 1
        i += 1
    pixelMatrix = temp[:]
    '''
    l = 0
    for f in range(height):
        for g in range(width):
            pixelMatrix[f][g][0]=(uint8)(temp[l][0])
            pixelMatrix[f][g][1]=(uint8)(temp[l][1])
            pixelMatrix[f][g][2]=(uint8)(temp[l][2])
            l += 1
    '''
    print("Done")

In [18]:
'''
#Choose Image and extract pixels matrix for the image
pixelMatrix = extractImgData("abc.jpeg")
height = len(pixelMatrix)
width = len(pixelMatrix[0])
print("Pixels Gathered")
#print(pixelMatrix[0,0,0]*2,pixelMatrix[0,0,0]*-2)
#Edits on the image
#Done...grayscale(height,width,pixelMatrix)
#Done...vertical_flip(height, width, pixelMatrix)
#Done...horizontal_flip(height, width, pixelMatrix)
#Done...edgify(height,width,pixelMatrix)
#Done...solarization(height,width,pixelMatrix,0)
#Done...brightness(height, width, pixelMatrix, 100)
#Done...contrast(height,width, pixelMatrix, 40)
#Done...blur(height, width, pixelMatrix,1)
#In-Built blur...pixelMatrix = cv2.blur(pixelMatrix,(5,5))
#m1 = time.time()
#blur(height,width,pixelMatrix,1)
#pixelMatrix = cv2.medianBlur(pixelMatrix,5)
#t = np.zeros((1,2,3))
#t[0,0] = [1,23,1]
#print(t)
#m2 = time.time()
#print("Image Edited:",m2-m1)
#Saving the image
imageio.imsave("nature-edited.jpeg", pixelMatrix);
print("Image Saved")
'''

'\n#Choose Image and extract pixels matrix for the image\npixelMatrix = extractImgData("abc.jpeg")\nheight = len(pixelMatrix)\nwidth = len(pixelMatrix[0])\nprint("Pixels Gathered")\n#print(pixelMatrix[0,0,0]*2,pixelMatrix[0,0,0]*-2)\n#Edits on the image\n#Done...grayscale(height,width,pixelMatrix)\n#Done...vertical_flip(height, width, pixelMatrix)\n#Done...horizontal_flip(height, width, pixelMatrix)\n#Done...edgify(height,width,pixelMatrix)\n#Done...solarization(height,width,pixelMatrix,0)\n#Done...brightness(height, width, pixelMatrix, 100)\n#Done...contrast(height,width, pixelMatrix, 40)\n#Done...blur(height, width, pixelMatrix,1)\n#In-Built blur...pixelMatrix = cv2.blur(pixelMatrix,(5,5))\n#m1 = time.time()\n#blur(height,width,pixelMatrix,1)\n#pixelMatrix = cv2.medianBlur(pixelMatrix,5)\n#t = np.zeros((1,2,3))\n#t[0,0] = [1,23,1]\n#print(t)\n#m2 = time.time()\n#print("Image Edited:",m2-m1)\n#Saving the image\nimageio.imsave("nature-edited.jpeg", pixelMatrix);\nprint("Image Saved")\n

In [19]:
'''
def start():
  # progbar.start(100)
  # progbar['value'] +=20
  # lab.config(text=progbar['value'])
  for x in range(5):
    progbar['value'] += 20
    root.update_idletasks()
    time.sleep(1)
    lab.config(text=progbar['value'])

# //// threading




def stop():
  progbar.stop()

progbar = ttk.Progressbar(root, length=100, orient=HORIZONTAL, mode='determinate')
progbar.pack()

lab = Label(root, text="")
lab.pack()

butt1 = Button(root, text="start", command=start)
butt1.pack()
butt2 = Button(root, text="stop", command=stop)
butt2.pack()
'''

'\ndef start():\n  # progbar.start(100)\n  # progbar[\'value\'] +=20\n  # lab.config(text=progbar[\'value\'])\n  for x in range(5):\n    progbar[\'value\'] += 20\n    root.update_idletasks()\n    time.sleep(1)\n    lab.config(text=progbar[\'value\'])\n\n# //// threading\n\n\n\n\ndef stop():\n  progbar.stop()\n\nprogbar = ttk.Progressbar(root, length=100, orient=HORIZONTAL, mode=\'determinate\')\nprogbar.pack()\n\nlab = Label(root, text="")\nlab.pack()\n\nbutt1 = Button(root, text="start", command=start)\nbutt1.pack()\nbutt2 = Button(root, text="stop", command=stop)\nbutt2.pack()\n'

In [20]:
root = tk.Tk()

root.title("Signals and Systems project")
# root.iconbitmap('C:/Users/microsoft/OneDrive/Desktop/camera.ico')
root.geometry("1020x1080")
root.configure(background="#212529")
#pixelMatrix = np.array()

def imgPreview():
    global myimage
    global img1
    global pixelMatrix
    global height
    global width

    root.filename = filedialog.askopenfilename(initialdir="C:/",
                                               title="Open Image"
                                               , filetypes=(("jpg files", "*.jpg"), ("png files", "*.png") , ("all files", "*.*")))
    # label = Label(root, text=root.filename)
    # label.pack()
    img1=Image.open(root.filename)
    # img1 = img1.resize((200, 1400),Image.ANTIALIAS)
    img1.thumbnail((500,1080))
    myimage = ImageTk.PhotoImage(img1)
    canvas0.create_image(350, 300, anchor=CENTER, image=myimage)
    pixelMatrix = extractImgData(root.filename)
    height = len(pixelMatrix)
    width = len(pixelMatrix[0])
    #return pixelMatrix
    #print(height,width)


# def saveImage():
#     root.filename2 = filedialog.asksaveasfile(title="Save", filetypes=(('jpeg','*.jpg'),('png','*.png'),('all files','*.*'))
#                                               , defaultextension=(('jpeg','*.jpg'),('png','*.png'),('all files','*.*')))
#     # files = [('jpeg','*.jpg'),('png','*.png'),('all files','*.*')]
#     # file = asksaveasfile(filetypes = files, defaultextension=files)
#     # img2 =Image.register_save(root.filename)
#     img1.save(root.filename)

def ExportImgPreview():
    global myimage2
    global img2
    imageio.imsave("temp.png", pixelMatrix)
    root.filename = "temp.png"
    img2 = Image.open(root.filename)
    # img1 = img1.resize((200, 1400),Image.ANTIALIAS)
    img2.thumbnail((500, 1080))
    myimage2 = ImageTk.PhotoImage(img2)
    canvas1.create_image(350, 300, anchor=CENTER, image=myimage2)

def saveimg():
    # filename1 = root.filename
    # img2 = Image.open(filename1)
    root.filename2 = filedialog.asksaveasfile(title="Save",
                                              filetypes=(('jpeg', '*.jpg'), ('png', '*.png'), ('all files', '*.*'))
                                              , defaultextension=(('jpeg', '*.jpg')))#, ('png', '*.png'), ('all files', '*.*')))
    #img2.save(root.filename2)
    #print(root.filename2)
    imageio.imsave(root.filename2.name, pixelMatrix)
    #('jpeg', '*.jpg'),
    # 350, 4200
    # imglabel = Label(root, image=myimage)
    # imglabel.grid(row=0,column=1, columnspan=4)
# my_img = ImageTk.PhotoImage(Image.open("Deadpool.png"))
# my_label = Label(root, image=my_img, width=300, height =300)
# my_label.grid(row=8, column=0)

# def gradButt():
#     class Example(tk.Frame):
#         def __init__(self, parent):
#             tk.Frame.__init__(self, parent)
#             f1 = GradientFrame(self, borderwidth=1, relief="sunken")
#             f2 = GradientFrame(self, "green", "blue", borderwidth=1, relief="sunken")
#             f1.pack(side="top", fill="both", expand=True)
#             f2.pack(side="bottom", fill="both", expand=True)
#
#     class GradientFrame(tk.Canvas):
#         '''A gradient frame which uses a canvas to draw the background'''
#
#         def __init__(self, parent, color1="red", color2="black", **kwargs):
#             tk.Canvas.__init__(self, parent, **kwargs)
#             self._color1 = color1
#             self._color2 = color2
#             self.bind("<Configure>", self._draw_gradient)
#
#         def _draw_gradient(self, event=None):
#             '''Draw the gradient'''
#             self.delete("gradient")
#             width = self.winfo_width()
#             height = self.winfo_height()
#             limit = width
#             (r1, g1, b1) = self.winfo_rgb(self._color1)
#             (r2, g2, b2) = self.winfo_rgb(self._color2)
#             r_ratio = float(r2 - r1) / limit
#             g_ratio = float(g2 - g1) / limit
#             b_ratio = float(b2 - b1) / limit
#
#             for i in range(limit):
#                 nr = int(r1 + (r_ratio * i))
#                 ng = int(g1 + (g_ratio * i))
#                 nb = int(b1 + (b_ratio * i))
#                 color = "#%4.4x%4.4x%4.4x" % (nr, ng, nb)
#                 self.create_line(i, 0, i, height, tags=("gradient",), fill=color)
#             self.lower("gradient")
#
#     if __name__ == "__main__":
#         root = tk.Tk()
#         Example(root).pack(fill="both", expand=True)


# def addImage():
#     # that app path which we displayed on frame is repeating so will fix that
#     # here widget is anything in the frame
#     # winfo_children() returns a list of all widgets which are children of this widget.
#     # for widget in frame.winfo_children():
#     #     widget.destroy()
#
#     # below code is to open a filedialogue on clicking the button
#     # in which this function is added (by command = function name)
#     filename = filedialog.askopenfilename( title="Select File",
#                                            filetypes=(("JPEG", "*.jpg"), ("all files", "*.*")) )
#     img1 = ImageTk.PhotoImage(Image.open(filename))
#     canvas1.create_image(root, 50,10, image=img1, anchor=NW)

    #  here apps(here images) path is obtained and shown


# here rightnow I am copying the code from which I am learning but I will make this function to
# open image in the aap get preview of the image in the app
# def runApps():
#     for app in apps:
#         os.startfile(app)

# canvas3 = tk.Canvas(root, height=750, width=170, bg="#ef476f")
# canvas3.grid(row=0,column=0, columnspan=1, rowspan=15, padx=0, pady=0, ipadx=0, ipady=0)


# ROW 0
# canvas1
# 457b9d

#  ////    ROW 0
canvas0 = tk.Canvas(root, height=600, width=700, bg="#457b9d")
canvas0.grid(row=0,column=1, columnspan=2, rowspan=14, padx=0, pady=0, ipadx=0, ipady=43)

# button  openImage
openImageButton = tk.Button(root, text="OPEN IMAGE", padx=10, pady=0, fg="black", bg="#d3d3d3", command=lambda:imgPreview())
openImageButton.grid(row=17,column=1, rowspan=1, columnspan=1, sticky=NW, padx=35, pady=20, ipadx=0, ipady=5)
#print(height,width,pixelMatrix)

# button  VignetteButton
GradientButton = tk.Button(root, text="VIGNETTE", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:vignette(height,width,pixelMatrix))
GradientButton.grid(row=1, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  EdgeDetectionButton
EdgeDetectionButton = tk.Button(root, text="EDGE DETECTION", padx=10, pady=0, fg="white", bg="#FF8811", command=lambda:edgify(height,width,pixelMatrix))
#threading.Thread(target = lambda:edgify(height,width,pixelMatrix)).start()
EdgeDetectionButton.grid(row=2, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  GrayscaleButton
GrayscaleButton = tk.Button(root, text="GRAYSCALE", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:grayscale(height,width,pixelMatrix))
GrayscaleButton.grid(row=3, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  HorizontalFlipButton
HorizontalFlipButton = tk.Button(root, text="VERTICAL FLIP", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:horizontal_flip(height,width,pixelMatrix))
HorizontalFlipButton.grid(row=4, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)

#  ////  ROW 1

canvas1 =tk.Canvas(root,height=600, width=700, bg="#264653")
canvas1.grid(row=0,column=3, columnspan=2, rowspan=14, padx=0, pady=0, ipadx=0, ipady=43)
# button  VerticalFlipButton
VerticalFlipButton = tk.Button(root, text="HORIZONTAL FLIP", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:vertical_flip(height,width,pixelMatrix))
VerticalFlipButton.grid(row=5, column=0, sticky=NW, padx=35, pady=5, ipadx=0, ipady=0)
# slider  solarisation
slide_solarisation= Scale(root, from_=0, to=255, orient=HORIZONTAL)
slide_solarisation.grid(row=6, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  brightness
slide_brightness= Scale(root, from_=1, to=100, orient=HORIZONTAL)
slide_brightness.grid(row=8, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  contrast
slide_contrast= Scale(root, from_=1, to=100, orient=HORIZONTAL)
slide_contrast.grid(row=10, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  blur
slide_blur= Scale(root, from_=1, to=4, orient=HORIZONTAL)
slide_blur.grid(row=12, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# # //// ROW 3
# canvas3 =tk.Canvas(root,height=50, width=820, bg="#264653")
# canvas3.grid(row=3, column=0,  columnspan=5, rowspan=1)
#
# label_solarisation = Label(root, text="Solarisation")
# label_solarisation.grid(row= 3, column=0 )
#
# label_brightness = Label(root, text="Brightness")
# label_brightness.grid(row= 3, column=1)
#
# label_contrast = Label(root, text="Contrast")
# label_contrast.grid(row= 3, column=2)
#
# label_blur = Label(root, text="Blur")
# label_blur.grid(row= 3, column=3)


#label1 = Label(root, text= slide_solarisation.get())
'''
def slide1():
    global label1 
    label1 = Label(root, text= slide_solarisation.get())
    print(slide_solarisation.get())
    # label1.grid(row=4, column=0)
def slide2():
    global label2 
    label2 = Label(root, text= slide_brightness.get())
    # label.grid(row=4, column=1)
def slide3():
    global label3 
    label3 = Label(root, text= slide_contrast.get())
    # label.grid(row=4, column=2)
def slide4():
    global label4 
    label4 = Label(root, text= slide_blur.get())
    # label.grid(row=4, column=3)
'''


label = Label(root, text= slide_solarisation.get())
slide_button = Button(root, text="SOLARIZATION",command=lambda:solarization(height,width,pixelMatrix,slide_solarisation.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=7, column=0, sticky="", padx=35, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_brightness.get())
slide_button = Button(root, text="BRIGHTNESS",command=lambda:brightness(height,width,pixelMatrix,slide_brightness.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=9, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_contrast.get())
slide_button = Button(root, text="CONTRAST",command=lambda:contrast(height,width,pixelMatrix,slide_contrast.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=11, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_blur.get())
slide_button = Button(root, text="BLUR",command=lambda:blur(height,width,pixelMatrix,slide_blur.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=13, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)



# canvas6 = tk.Canvas(root, height=200, width=700, bg="#457b9d")
# canvas6.grid(row=6, column=1, columnspan=4)

# button 8 Export
Export = tk.Button(root, text="EXPORT", padx=0, pady=0, fg="black", bg="#d3d3d3", command=saveimg)
Export.grid(row=17, column=2, sticky="", padx=14, pady=20, ipadx=5, ipady=5)

# //// ROW 7
# button 9 Exit
Exit = tk.Button(root, text="EXIT",padx=0, pady=0, fg="black", bg="#d3d3d3", command=root.destroy)
Exit.grid(row=17, column=4, sticky="", padx=0, pady=20, ipadx=0, ipady=5)

#  button Preview
preview = Button(root, text="PREVIEW", command=ExportImgPreview,padx=0, pady=0, fg="black", bg="#d3d3d3")
preview.grid(row=17, column=3, sticky="", padx=0, pady=20, ipadx=0, ipady=5)

# frame if you want
# (relwidth, relheight, relx, rely if they are 1 then full canvas will  be occupied)
# frame = tk.Frame(root, bg="white")
# frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)
root.mainloop()
img1.close()
img2.close()
os.remove("temp.png")